In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from openpyxl import load_workbook
import os

# --- CONFIGURAÇÕES ---
EMAIL_REMETENTE = "caioguilherme137@gmail.com"
SENHA_APP = "ihzc lple tdde ywoy"
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465
PASTA_ANEXOS = r"C:\Users\caioz\Desktop\curso rpa"
PLANILHA_EMAILS = os.path.join(PASTA_ANEXOS, "ListaEmail.xlsx")

# --- ABRIR PLANILHA ---
wb = load_workbook(PLANILHA_EMAILS)
aba = wb.active

# --- CONECTAR AO GMAIL ---
with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as smtp:
    smtp.login(EMAIL_REMETENTE, SENHA_APP)

    for linha in aba.iter_rows(min_row=2, values_only=True):
        _, nome, email_destinatario = linha[:3]  # supondo colunas A, B, C

        if not nome or not email_destinatario:
            continue

        # --- MONTAR E-MAIL ---
        msg = MIMEMultipart()
        msg["From"] = EMAIL_REMETENTE
        msg["To"] = email_destinatario
        msg["Subject"] = f"Relatório Personalizado para {nome}"

        corpo = f"Olá, {nome}.\n\nEste é um e-mail de teste com seu anexo personalizado.\n\nAtenciosamente."
        msg.attach(MIMEText(corpo, "plain"))

        # --- ANEXO ---
        caminho_anexo = os.path.join(PASTA_ANEXOS, f"{nome}.xlsx")
        if os.path.exists(caminho_anexo):
            with open(caminho_anexo, "rb") as f:
                part = MIMEBase("application", "octet-stream")
                part.set_payload(f.read())
            encoders.encode_base64(part)
            part.add_header("Content-Disposition", f"attachment; filename={os.path.basename(caminho_anexo)}")
            msg.attach(part)
        else:
            print(f"⚠️ Anexo não encontrado: {caminho_anexo}")

        # --- ENVIAR ---
        smtp.sendmail(EMAIL_REMETENTE, email_destinatario, msg.as_string())
        print(f"✅ E-mail enviado para {email_destinatario}")

print("📨 Processo finalizado com sucesso!")
